### Import Packages

In [ ]:
import os
import io
import sys
import time
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import pandas as pd
import skimage
from operator import itemgetter
from skimage.filters.rank import entropy
from skimage.draw import disk
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.io import imread
from skimage.io import imread_collection
from scipy import ndimage as ndi
from skimage import feature, data, filters, measure, morphology
import plotly
import plotly.express as px
import plotly.graph_objects as go
import keyboard
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
%matplotlib qt
mpl.rcParams['savefig.dpi'] = 800

### Define Function: Calculate Mean Intensities from Detected Circles

In [ ]:
def mean_intensity(num_new_img, Apical, Basal, drawn_circles_list, ACTUAL_RADIUS, img_set_recent, img_set_base):
    """This takes inputs from the Locate_Fibers.ipynb and Photometry_Barrier_Function_Analysis.ipynb and returns 
    the mean grey intensity from each fiber identified"""
    
    #Creating empty lists for measured intensity
    intensity = []
    intensity_base = []
    center = []
    height, width = Apical[0].shape[:2]
    for arr in drawn_circles_list: #organizes the list of drawn circles from leftmost circle to rightmost
        arr.sort(key = itemgetter(1))
        
    for j in np.arange(len(drawn_circles_list)): #number of channels that we are iterating over (len = 2 when Apical and Basal channels are analyzed)
        mask = np.zeros((height, width)) #The black background we will use for our mask
        for center_y, center_x, radius in drawn_circles_list[j]: #Taking saved coordinates and radii from the processed image
            rr,cc = skimage.draw.disk((center_y, center_x), ACTUAL_RADIUS) #drawing disks using the saved coordinates
            mask[rr, cc] = 1 #saving those disks onto the mask with a uniform intensity of 1

        labels = measure.label(mask) #Grab the circles from the mask using nearby pixels with identical intensity (1)
        for z in np.arange(len(img_set_recent[j])):
            for n in np.arange(len(img_set_recent[j,z])): #Looping over all images, for each of the 7 measured circles in each image
                for k in np.arange(len(drawn_circles_list[j])):
                    props = measure.regionprops(labels, intensity_image=img_set_recent[j,z,n])#Measure properties of the original image using the 7 labeled circles from the mask
                    intensity_val = props[k]['intensity_mean']  #Taking the individual circle property (7 per image) 
                    intensity.append(intensity_val) #Appending individual circle images from each image one at a time
        
        print(100*(j+1)/len(drawn_circles_list),'% complete')#ie. appending circle 1, image 1 / then circle 2, image 1 / ...
                      
        for q in np.arange(len(drawn_circles_list[j])):
            props = measure.regionprops(labels, intensity_image=img_set_recent[j,0,0])#Measure properties of the original image using the 7 labeled circles from the mask
            center_val = props[q]['centroid']  #Taking the individual circle property (7 per image)
            center.append(center_val)
    
        for x in np.arange(len(img_set_base[j])):
            for y in np.arange(len(drawn_circles_list[j])):
                props_b = measure.regionprops(labels, intensity_image=img_set_base[j,x])
                intensity_b = props_b[y]['intensity_mean']
                intensity_base.append(intensity_b)
    
    center_apical = center[:int(len(center)/2)]
    center_basal = center[int(len(center)/2):] 
    
    for i in np.arange(len(center_apical)):
        center_apical[i] = (center_apical[i][1])
        center_basal[i] = (center_basal[i][1])
    center_all = [center_apical] + [center_basal]      
    
    #Reshaping our output arrays for further analysis
    intensity_array_Apical = np.reshape(intensity[:int(len(intensity)/2)],(int((len(intensity)/2)/len(drawn_circles)), len(drawn_circles)))
    intensity_array_Basal = np.reshape(intensity[int(len(intensity)/2):],(int((len(intensity)/2)/len(drawn_circles)), len(drawn_circles)))
    intensity_array = np.array([intensity_array_Apical, intensity_array_Basal])
    
    #Further reshaping our output arrays for analysis
    reorder_Ap = np.insert(intensity_array[0], 0, center_all[0], axis = 0)
    reorder_Bas = np.insert(intensity_array[1], 0, center_all[1], axis = 0)
    reorder_Ap = reorder_Ap[:, reorder_Ap[0].argsort()]
    reorder_Bas = reorder_Bas[:, reorder_Bas[0].argsort()]
    reordered_array = np.array([reorder_Ap, reorder_Bas])
    intensity_array = reordered_array[:,1:,:]
    intensity_array_Apical = intensity_array[0]
    intensity_array_Basal = intensity_array[1]
    
    #Reshaping our output baseline arrays for further analysis
    intensity_array_base_Apical = np.reshape(intensity_base[:int(len(intensity_base)/2)],(int((len(intensity_base)/2)/len(drawn_circles)), len(drawn_circles)))
    intensity_array_base_Basal = np.reshape(intensity_base[int(len(intensity_base)/2):],(int((len(intensity_base)/2)/len(drawn_circles)), len(drawn_circles)))
    intensity_array_base = np.array([intensity_array_base_Apical, intensity_array_base_Basal])
    
    #Further reshaping our output baseline arrays for analysis
    reorder_Ap_B = np.insert(intensity_array_base[0], 0, center_all[0], axis = 0)
    reorder_Bas_B = np.insert(intensity_array_base[1], 0, center_all[1], axis = 0)
    reorder_Ap_B = reorder_Ap_B[:, reorder_Ap_B[0].argsort()]
    reorder_Bas_B = reorder_Bas_B[:, reorder_Bas_B[0].argsort()]
    reordered_array_B = np.array([reorder_Ap_B, reorder_Bas_B])
    intensity_array_base = reordered_array_B[:,1:,:]
    intensity_array_base_Apical = intensity_array_base[0]
    intensity_array_base_Basal = intensity_array_base[1]
    
    
    print('Measured mean intensities from', len(drawn_circles), 'detected fibers across', len(intensity_array_Apical),'most recent image(s) of', len(Apical), 'total images.')
    
    return intensity_array_Apical, intensity_array_Basal, intensity_array, intensity_array_base_Apical, intensity_array_base_Basal, intensity_array_base

### Define Function: Calculate DFF

In [ ]:
#combining the two calculated intensity arrays into a single, 3D array
#indexing: [apical or basal, image number, circle number]
#intensity_array = np.array([intensity_array_Apical, intensity_array_Basal])

def DFF(intensity_array, record_rate, intensity_array_base, num_new_img, Apical):
    """This function will take the analyzed output intensity values from the Apical and Basal channels
    #and ouput DFF values"""
    
    IMG = intensity_array   #the newly analyzed images from mean_intensity
    Base = intensity_array_base[:,1:10,:] #Using images 2 to 10 for baseline since image 1 will be used to detect circles
    Basemean = np.array([np.mean(Base[0,:,:], axis = 0), np.mean(Base[1,:,:], axis = 0)], dtype=float) #calculating the mean baseline value across the baseline images
    
    time_array = [] #setting a blank time array to fill in with times when new images where taken 
    time_set = np.arange(len(Apical[10:])) #Apical from Locate_Fibers.ipynb, full set of time from experiment start to most recent image taken
    
    if len(Apical) >= 20: #limit the analysis to minimum 20 collected images
        #for the number of new images, calculate the experiment time that each was taken
        for i in time_set:    
            Time = i*record_rate/60 #hrs
            time_array.append(Time)
    time_array = time_array[-num_new_img:]
    
    dff = [] #Empty set to contain our calculated deltaF/F values
    for k in np.arange(len(IMG)):
        for h in np.arange(len(IMG[0])):
            for j in np.arange(len(Basemean[0])): #The 7 means for each of the 7 circles
                dff.append((IMG[k,h,j]-Basemean[k,j])/Basemean[k,j])
    dff = np.reshape(dff,(len(IMG),len(IMG[0]),len(IMG[0,0])))
    
    return dff, time_array

### Define Function: Continuously Update Excel for Plotting

In [ ]:
def update_excel(time_array, dff, Apical, intensity_array):
    """ This function outputs an excel file with the timestamped dF/F values and raw mean grey intensity values
    the file path chosen in Photometry_Barrier_Function_Analysis.ipynb will be used as the location to output the excel file"""
    
    sheets = ['Apical', 'Basal']
    df = pd.ExcelFile(f"{folder}.xlsx")
    
    for k in np.arange(len(sheets)):
        df_all = pd.read_excel(df, sheets[k], index_col = False, header = None, engine="openpyxl")
        time_col = df_all.iloc[0:,0]
        time_col = time_col.to_numpy()
       
        compiled_col = np.column_stack((time_array, dff[k], intensity_array[k]))
        
        dfout = pd.DataFrame(compiled_col)
        sheet = sheets[k]
        with pd.ExcelWriter(f"{folder}.xlsx", mode = 'a', if_sheet_exists = 'overlay', engine = 'openpyxl') as writer:
            dfout.to_excel(writer, sheet_name = sheet, index = False, header = False, startrow = len(time_col), na_rep = 'nan')

    print('From a total of', len(Apical), 'captured images,',len(time_array), 'new image(s) were analyzed.')

#### Define Function: Plotted Output

In [ ]:
def Plot_all():
    """This function will output a plot of dF/F values calculated for all active fiber ends as a function of time"""

    %store -r folder 
    #recover folder path from Photometry_Barrier_Function_Analysis.ipynb
    %store -r EXPECTED_FIBERS 
    #recover the number of fibers expexted from Photometry_Barrier_Function_Analysis.ipynb

    sheets = ['Apical', 'Basal'] #names of excel sheets within the excel file of interest

    with open(f"{folder}.xlsx", "rb") as f: #dataframe/excel file to pull data from
        df = io.BytesIO(f.read())
    
    df_api = pd.read_excel(df, sheets[0], index_col = False, header = None, engine="openpyxl") #pulling apical data from the 'Apical' sheet
    df_bas = pd.read_excel(df, sheets[1], index_col = False, header = None, engine="openpyxl") #pulling basal data from the 'Basal' sheet

    t_axis_api = df_api.iloc[1:,0] #grabbing each of the time columns from the dataframes
    t_axis_api = t_axis_api.to_numpy()
    t_axis_bas = df_bas.iloc[1:,0] #grabbing each of the time columns from the dataframes
    t_axis_bas = t_axis_bas.to_numpy()
    #grabbing the remaining columns, from column 1 to the last one (3 columns for 3 fibers/7 columns for 7 fibers/etc.)
    #from apical and basal sheets
    api_lines = df_api.iloc[1:,1:(EXPECTED_FIBERS+1)]
    api_lines = api_lines.to_numpy()
    bas_lines = df_bas.iloc[1:,1:(EXPECTED_FIBERS+1)]
    bas_lines = bas_lines.to_numpy()

    all_lines = np.column_stack((api_lines, bas_lines))

    fig = plt.figure(figsize = (36,20)) #initialized the plot
    ax = plt.axes()
    cmap = mpl.cm.BrBG
    cmap1 = mpl.cm.PiYG(np.linspace(0.0,0.35,128))
    cmap2 = mpl.cm.PiYG_r(np.linspace(0.0,0.35,128))
    cmapboth = np.vstack((cmap1,cmap2))
    cmap = mcolors.LinearSegmentedColormap.from_list('HalfHalf', cmapboth) 
    ax.set_title('Fluorescence Signal over Time', fontsize = 30)
    ax.set_xlabel('Time (hr)', fontsize = 30)
    ax.set_ylabel('$\\Delta$F/F', fontsize = 30)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 25, length = 7, width = 3)
    

    N = len(all_lines[0]) #total number of fibers to plot
    name_labels = [] #gathering a list of legend labels
    for i in sheets: #grabbing from the excel sheet name
        for j in range(int(N/2)): #splitting the number of total fibers in half to count 1,2,3 apical then back to 1,2,3 basal instead of 1 to 6
            name_labels.append(f'%s {j+1}' %i) 

    [plt.plot(t_axis_api, all_lines[:,_], color = cmap(_/(N-1)), linestyle = '-', linewidth = 2, marker = 'o', markersize = 4, label = name_labels[_])[0] for _ in range(N)] #lines to animate, do all line formatting here
    plt.legend(title = 'Channel and Fiber Number', ncol = len(sheets), framealpha = 0, fontsize = 25, title_fontsize = '25')
    return(fig)

### Define Function: Automated Analysis

In [ ]:
def automation_cycle():
    """Takes all the prior functions and runs them for full analysis and output of sensed photometry data
    Requires at minimum 20 images (10 for baseline, 10 for experimental) for robust data analysis"""
    
    print(time.strftime("Starting analysis at: %H:%M:%S | %b %d %Y"))
    df = pd.ExcelFile(f"{folder}.xlsx")

    new_img_comp = pd.read_excel(df, 'Apical', index_col = False, header = None, engine="openpyxl")
    new_img_col = new_img_comp.iloc[0:,0]
    new_img_col = new_img_col.to_numpy()
    
    Apical = imread_collection(dirNameA, conserve_memory=True) #This will update every 10 minutes to include new images
    Basal = imread_collection(dirNameB, conserve_memory=True)  #This will update every 10 minutes to include new images
            
    num_baseline_img = 10
    num_new_img = len(Apical) - (len(new_img_col) + num_baseline_img - 1)
                
    if num_new_img > 0 and len(Apical) >= 20 and len(Apical) == len(Basal): #analyzes the data in chunks to avoid slow load times
        print('New images found, starting image processing.')
        def divisors(n): #divides the data into equally divided chunks for analysis
            divisors = []
            for i in range(1, int(n**0.5)+1):
                if n % i == 0:
                    divisors.append(i)
                    if i != n // i:
                        divisors.append(n // i)
            return divisors
                
        split_num_all = divisors(len(Apical[-num_new_img:]))
        split_num = split_num_all[0]
                
        Api_split = np.array_split(Apical[-num_new_img:], split_num)        
        Bas_split = np.array_split(Basal[-num_new_img:], split_num)
                
        img_set_recent = np.array([Api_split, Bas_split])
            
        img_set_base = np.array([Apical[0:20], Basal[0:20]], dtype = object)
        print(f'Total memory (RAM) required: {sys.getsizeof(img_set_recent)/1e9:.2f} GB')
        intensity_array_Apical, intensity_array_Basal, intensity_array, intensity_array_base_Apical, intensity_array_base_Basal, intensity_array_base = mean_intensity(num_new_img, Apical, Basal, drawn_circles_list, ACTUAL_RADIUS, img_set_recent, img_set_base)          
        dffarr, Time = DFF(intensity_array, record_rate, intensity_array_base, num_new_img, Apical)
        update_excel(Time, dffarr, Apical, intensity_array)
                
    elif len(Apical) < 20:
        print('Not enough images collected for analysis:', (len(Apical) - 1), 'of 20 minimum images collected.')
    else:
        print('No new images acquired.')
    Plot_all()
    print(time.strftime("Completed: %H:%M:%S | %b %d %Y \n---------------------------------------------------"))